# Testing & Quality Overview

A comprehensive guide to backend testing strategies, test doubles, and quality assurance practices.

---

## 1. The Testing Pyramid

The testing pyramid guides the **distribution** of tests across different levels:

```
                    /\
                   /  \
                  / E2E \           ← Slow, Expensive, Few
                 /  Tests \            (UI, Full System)
                /──────────\
               / Integration \      ← Medium Speed, Some
              /    Tests      \        (APIs, DB, Services)
             /─────────────────\
            /    Unit Tests     \   ← Fast, Cheap, Many
           /     (70-80%)        \     (Functions, Classes)
          /───────────────────────\
```

| Level | Scope | Speed | Cost | Coverage Target |
|-------|-------|-------|------|----------------|
| **Unit** | Single function/class | ~ms | Low | 70-80% |
| **Integration** | Multiple components | ~seconds | Medium | 15-20% |
| **E2E** | Full system flow | ~minutes | High | 5-10% |

## 2. Test Doubles: Mocks, Stubs & Fakes

Test doubles replace real dependencies during testing:

```
┌─────────────────────────────────────────────────────────────────┐
│                      TEST DOUBLES                               │
├─────────────┬───────────────────────────────────────────────────┤
│   STUB      │ Returns canned responses, no verification        │
│             │ Example: Always return {"status": "ok"}          │
├─────────────┼───────────────────────────────────────────────────┤
│   MOCK      │ Verifies interactions (calls, args, count)       │
│             │ Example: Assert send_email() called once         │
├─────────────┼───────────────────────────────────────────────────┤
│   FAKE      │ Working implementation with shortcuts            │
│             │ Example: In-memory DB instead of PostgreSQL      │
├─────────────┼───────────────────────────────────────────────────┤
│   SPY       │ Real object that records interactions            │
│             │ Example: Track all method calls on real service  │
└─────────────┴───────────────────────────────────────────────────┘
```

**When to Use:**
- **Stub**: External APIs, slow services, when you don't care about calls
- **Mock**: Verify side effects (emails sent, events published)
- **Fake**: Complex dependencies (databases, file systems)

In [ ]:
# Example: Test Doubles with pytest and unittest.mock
from unittest.mock import Mock, patch, MagicMock
from dataclasses import dataclass
from typing import Protocol

# --- Production Code ---
@dataclass
class User:
    id: int
    email: str

class EmailService(Protocol):
    def send(self, to: str, subject: str, body: str) -> bool: ...

class UserService:
    def __init__(self, db_repo, email_service: EmailService):
        self.db = db_repo
        self.email = email_service
    
    def register(self, email: str) -> User:
        user = User(id=1, email=email)
        self.db.save(user)
        self.email.send(email, "Welcome!", "Thanks for joining.")
        return user

# --- Tests with Mocks/Stubs ---
def test_register_saves_user_and_sends_email():
    # Arrange: Create test doubles
    mock_db = Mock()                    # Mock for DB
    mock_email = Mock()                 # Mock for Email
    mock_email.send.return_value = True # Stub behavior
    
    service = UserService(mock_db, mock_email)
    
    # Act
    user = service.register("test@example.com")
    
    # Assert: Verify interactions (Mock assertions)
    mock_db.save.assert_called_once()
    mock_email.send.assert_called_once_with(
        "test@example.com", "Welcome!", "Thanks for joining."
    )
    assert user.email == "test@example.com"

# Run the test
test_register_saves_user_and_sends_email()
print("✅ Test passed: Mocks verified correctly!")

## 3. Code Coverage Strategies

```
┌────────────────────────────────────────────────────────────────┐
│                   COVERAGE METRICS                             │
├────────────────┬───────────────────────────────────────────────┤
│ Line Coverage  │ % of code lines executed                      │
│ Branch Coverage│ % of decision branches (if/else) taken        │
│ Path Coverage  │ % of execution paths through code             │
│ Function Cov.  │ % of functions/methods called                 │
└────────────────┴───────────────────────────────────────────────┘
```

**Best Practices:**

| Practice | Description |
|----------|-------------|
| Target 80%+ | Diminishing returns beyond 90% |
| Branch > Line | Branch coverage catches more edge cases |
| Exclude noise | Skip generated code, migrations, config |
| CI enforcement | Fail builds if coverage drops |

```bash
# Run pytest with coverage
pytest --cov=src --cov-report=html --cov-branch --cov-fail-under=80
```

In [ ]:
# Example: Pytest fixtures and parametrized tests
import pytest
from typing import List

# --- Code Under Test ---
def calculate_discount(price: float, tier: str) -> float:
    """Apply discount based on customer tier."""
    discounts = {"bronze": 0.05, "silver": 0.10, "gold": 0.20}
    if price < 0:
        raise ValueError("Price cannot be negative")
    return price * (1 - discounts.get(tier, 0))

# --- Parametrized Test for Branch Coverage ---
test_cases = [
    (100, "bronze", 95.0),   # 5% discount
    (100, "silver", 90.0),   # 10% discount
    (100, "gold", 80.0),     # 20% discount
    (100, "unknown", 100.0), # No discount
]

# Simulating pytest.mark.parametrize behavior
for price, tier, expected in test_cases:
    result = calculate_discount(price, tier)
    assert result == expected, f"Failed for {tier}"
    print(f"✅ {tier}: ${price} → ${result}")

# Test edge case
try:
    calculate_discount(-10, "gold")
except ValueError as e:
    print(f"✅ Caught expected error: {e}")

## 4. API Testing

```
┌─────────────────────────────────────────────────────────────────┐
│                    API TESTING LAYERS                           │
├─────────────────────────────────────────────────────────────────┤
│  CONTRACT TESTS   ←── Schema validation, OpenAPI compliance     │
│        ↓                                                        │
│  INTEGRATION      ←── Real HTTP calls to running service        │
│        ↓                                                        │
│  UNIT (Handler)   ←── Test route handlers in isolation          │
└─────────────────────────────────────────────────────────────────┘
```

**Key Testing Areas:**
- ✅ Status codes (200, 400, 401, 404, 500)
- ✅ Response schema validation
- ✅ Authentication & authorization
- ✅ Input validation & error messages
- ✅ Idempotency (for POST/PUT)

In [ ]:
# Example: API Testing with FastAPI TestClient
# (Simulated without actual FastAPI import)

from dataclasses import dataclass
from typing import Optional

# Simulated API response class
@dataclass
class Response:
    status_code: int
    json_data: dict
    
    def json(self):
        return self.json_data

# Simulated test client
class MockTestClient:
    def get(self, path: str, **kwargs) -> Response:
        if path == "/users/1":
            return Response(200, {"id": 1, "name": "Alice"})
        return Response(404, {"detail": "Not found"})
    
    def post(self, path: str, json: dict, **kwargs) -> Response:
        if not json.get("email"):
            return Response(422, {"detail": "Email required"})
        return Response(201, {"id": 1, **json})

# --- API Tests ---
client = MockTestClient()

def test_get_user_success():
    response = client.get("/users/1")
    assert response.status_code == 200
    assert response.json()["name"] == "Alice"
    print("✅ GET /users/1 → 200 OK")

def test_get_user_not_found():
    response = client.get("/users/999")
    assert response.status_code == 404
    print("✅ GET /users/999 → 404 Not Found")

def test_create_user_validation():
    response = client.post("/users", json={})
    assert response.status_code == 422  # Validation error
    print("✅ POST /users (invalid) → 422 Validation Error")

def test_create_user_success():
    response = client.post("/users", json={"email": "bob@test.com", "name": "Bob"})
    assert response.status_code == 201
    assert "id" in response.json()
    print("✅ POST /users → 201 Created")

# Run all tests
test_get_user_success()
test_get_user_not_found()
test_create_user_validation()
test_create_user_success()

## 5. Load Testing Basics

```
┌─────────────────────────────────────────────────────────────────┐
│                   LOAD TESTING TYPES                            │
├──────────────┬──────────────────────────────────────────────────┤
│ Smoke        │ Minimal load to verify system works             │
│ Load         │ Expected concurrent users (baseline)            │
│ Stress       │ Beyond capacity → find breaking point           │
│ Spike        │ Sudden traffic bursts                           │
│ Soak/Endurance│ Sustained load over hours → memory leaks       │
└──────────────┴──────────────────────────────────────────────────┘

        Response Time vs Load
        │
  Time  │              ╱╱
  (ms)  │            ╱╱
        │          ╱╱   ← Degradation point
        │    ─────╱
        │─────────
        └────────────────────→ Concurrent Users
```

**Key Metrics:**
- **Throughput**: Requests per second (RPS)
- **Latency**: p50, p95, p99 response times
- **Error Rate**: % of failed requests
- **Saturation**: CPU, memory, connection pool usage

**Tools**: Locust (Python), k6, JMeter, Gatling

## 6. TDD Workflow (Test-Driven Development)

```
          ┌─────────────────────────────────────┐
          │          TDD CYCLE                  │
          │                                     │
          │    ┌─────────┐                      │
          │    │  RED    │ Write failing test   │
          │    └────┬────┘                      │
          │         ↓                           │
          │    ┌─────────┐                      │
          │    │  GREEN  │ Make it pass         │
          │    └────┬────┘ (minimal code)       │
          │         ↓                           │
          │    ┌─────────┐                      │
          │    │REFACTOR │ Improve design       │
          │    └────┬────┘                      │
          │         │                           │
          │         └──────→ Repeat             │
          └─────────────────────────────────────┘
```

**TDD Benefits:**
1. **Design feedback** - Forces modular, testable code
2. **Documentation** - Tests describe expected behavior
3. **Confidence** - Refactor without fear
4. **Focus** - One small step at a time

In [ ]:
# TDD Example: Building a Stack data structure

# Step 1: RED - Write failing tests first
def run_stack_tests(StackClass):
    """Test suite for Stack implementation."""
    
    # Test: Empty stack
    stack = StackClass()
    assert stack.is_empty() == True
    assert len(stack) == 0
    print("✅ Empty stack works")
    
    # Test: Push and peek
    stack.push(1)
    assert stack.is_empty() == False
    assert stack.peek() == 1
    print("✅ Push and peek work")
    
    # Test: Pop returns last pushed
    stack.push(2)
    stack.push(3)
    assert stack.pop() == 3
    assert stack.pop() == 2
    print("✅ Pop returns LIFO order")
    
    # Test: Pop empty raises
    stack.pop()  # Remove last item
    try:
        stack.pop()
        assert False, "Should have raised"
    except IndexError:
        print("✅ Pop empty raises IndexError")

# Step 2: GREEN - Implement minimal code to pass
class Stack:
    def __init__(self):
        self._items = []
    
    def push(self, item):
        self._items.append(item)
    
    def pop(self):
        if self.is_empty():
            raise IndexError("Pop from empty stack")
        return self._items.pop()
    
    def peek(self):
        return self._items[-1]
    
    def is_empty(self):
        return len(self._items) == 0
    
    def __len__(self):
        return len(self._items)

# Step 3: Run tests (GREEN) → Then REFACTOR if needed
run_stack_tests(Stack)
print("\n🎉 All TDD tests passed!")

## Quick Reference

| Topic | Key Takeaway |
|-------|-------------|
| **Testing Pyramid** | Many unit tests, fewer integration, minimal E2E |
| **Mocks** | Verify interactions happened |
| **Stubs** | Provide canned responses |
| **Fakes** | Lightweight working implementations |
| **Coverage** | Target 80%+, prefer branch over line |
| **API Testing** | Test status codes, validation, auth |
| **Load Testing** | Measure p95/p99 latency, find breaking point |
| **TDD** | Red → Green → Refactor cycle |

---

**Further Reading:**
- [pytest Documentation](https://docs.pytest.org/)
- [Locust Load Testing](https://locust.io/)
- [Test Doubles by Martin Fowler](https://martinfowler.com/bliki/TestDouble.html)